In [10]:
# Import the dependencies
import os
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt

# Define directory containing the csv files
dir_path = Path("../Resources")

# create an empty list to hold the dataframes 
dfs = []

# loop through each file in the directory
for file in os.listdir(dir_path):
    # check if the file is a csv file
    if file.endswith(".csv"):
        # extract the ticker symbol from the file name (assuming the file name is the ticker symbol)
        ticker = file.replace(".csv", "")
        # Read the file into a DataFrame
        stocks_df = pd.read_csv(dir_path / file)
        # Add a column to the DataFrame to store the ticker symbol
        stocks_df["Ticker"] = ticker
        # add the dataframe to the list
        dfs.append(stocks_df)
        
# concatenate the dataframes in the list
combined_stocks_df = pd.concat(dfs, ignore_index=True)

# Display the combined DataFrame to verify
print(combined_stocks_df.head())

         Date  Open      High       Low     Close  Adj Close     Volume Ticker
0  1990-02-16   0.0  0.079861  0.073785  0.077257   0.052392  940636800   CSCO
1  1990-02-20   0.0  0.079861  0.074653  0.079861   0.054158  151862400   CSCO
2  1990-02-21   0.0  0.078993  0.075521  0.078125   0.052980   70531200   CSCO
3  1990-02-22   0.0  0.081597  0.078993  0.078993   0.053569   45216000   CSCO
4  1990-02-23   0.0  0.079861  0.078125  0.078559   0.053275   44697600   CSCO


/var/folders/f4/7lkrmhr91498yvvn03jsc52c0000gp/T/ipykernel_3378/3938572439.py:35: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_stocks_df = pd.concat(dfs, ignore_index=True)


In [11]:
# view the shape of the data
print(combined_stocks_df.shape)

(456778, 8)


In [12]:
# change the type in 'date' column to datetime
combined_stocks_df["Date"] = pd.to_datetime(combined_stocks_df["Date"])

# Filter the data for the last 5 years from today
five_years_ago = pd.Timestamp.today() - pd.DateOffset(years=5)
stocks_data_filtered = combined_stocks_df[combined_stocks_df["Date"] >= five_years_ago]

print(stocks_data_filtered.head())
print(stocks_data_filtered.tail())



           Date       Open       High        Low      Close  Adj Close  \
7382 2019-06-07  55.410000  56.299999  55.029999  55.930000  47.970558   
7383 2019-06-10  56.139999  56.650002  55.959999  56.419998  48.390820   
7384 2019-06-11  56.740002  57.560001  56.730000  57.110001  48.982639   
7385 2019-06-12  56.049999  56.590000  55.650002  55.860001  47.910522   
7386 2019-06-13  55.919998  56.270000  55.779999  56.169998  48.176399   

        Volume Ticker  
7382  17394900   CSCO  
7383  16871000   CSCO  
7384  26723700   CSCO  
7385  23183000   CSCO  
7386  15785300   CSCO  
             Date        Open        High         Low       Close   Adj Close  \
456773 2024-04-17  950.000000  952.489990  910.260010  912.260010  912.260010   
456774 2024-04-18  909.229980  909.330017  884.390015  888.919983  888.919983   
456775 2024-04-19  894.940002  899.679993  861.950012  870.250000  870.250000   
456776 2024-04-22  876.400024  878.739990  857.979980  868.070007  868.070007   
456777

In [13]:
# Initialize a dictionary to hold the MSE for each stock symbol
mse_dict = {}

# Initialize a dictionary to hold the predictions for each stock symbol
predictions_dict = {}

# Group the data by ticker symbol
stocks_grouped = stocks_data_filtered.groupby("Ticker")

# # Iterate through each group (stock symbol)
for ticker, group in stocks_grouped:
    # Create a copy of the group's data
    group_copy = group.copy()
    
    # Set the date as the index
    group_copy.set_index("Date", inplace=True)
    
    # Define a feature int he data for previous date closing prices
    group_copy["Previous Day Close"] = group_copy["Close"].shift(1)
    
    # Define a feature in the data for the volume
    group_copy["Volume Difference"] = group_copy["Volume"].diff()
    
    # Drop rows with NaN values
    group_copy = group_copy.dropna()
    
    # Define the features (X) and the target (y) variables for training purposes
    X = group_copy[["Previous Day Close", "Volume Difference"]]
    y = group_copy["Close"].values.reshape(-1, 1) 
    
    # split the data into training and testing sets chronologically
    split = int(0.7 * len(X))
    X_train = X[: split]
    X_test = X[split:]
    y_train = y[: split]
    y_test = y[split:]
    
    # Train the model
    model = LinearRegression()
    model.fit(X_train, y_train)
    
    # Make predictions using the testing data
    predictions = model.predict(X_test)
    
    # Calculate the mean_squared_error on the testing data
    mse = mean_squared_error(y_test, predictions)
    
    # Store the MSE in the dictionary
    mse_dict[ticker] = mse
    
        # Make a prediction for the values of the stock over the next year starting from the last date for each stock in the dataset
    # Create a dataframe to hold the predictions
    future_dates = pd.date_range(start=group_copy.index[-1], periods=12, freq='M')
    future_dates_df = pd.DataFrame(future_dates, columns=["Date"])
    future_dates_df.set_index("Date", inplace=True)
    
    # Initialize previous_close with the last available close value
    previous_close = group_copy["Close"].iloc[-1]
    volume_difference = 0

    # Predict the future values iteratively
    future_closes = []
    for date in future_dates_df.index:
        # prepare the input data for prediction
        input_data = pd.DataFrame({"Previous Day Close": [previous_close], "Volume Difference": [volume_difference]})
        # Make the prediction
        predicted_close = model.predict(input_data)[0][0]
        # Append the predicted close to the list
        future_closes.append(predicted_close)
        # Update previous_close for the next iteration
        previous_close = predicted_close
    
    # Add the predictions to the dataframe
    future_dates_df["Close"] = future_closes
    
    # Store the predictions in the dictionary
    predictions_dict[ticker] = future_dates_df

/var/folders/f4/7lkrmhr91498yvvn03jsc52c0000gp/T/ipykernel_3378/855425967.py:53: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=group_copy.index[-1], periods=12, freq='M')
/var/folders/f4/7lkrmhr91498yvvn03jsc52c0000gp/T/ipykernel_3378/855425967.py:53: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=group_copy.index[-1], periods=12, freq='M')
/var/folders/f4/7lkrmhr91498yvvn03jsc52c0000gp/T/ipykernel_3378/855425967.py:53: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=group_copy.index[-1], periods=12, freq='M')
/var/folders/f4/7lkrmhr91498yvvn03jsc52c0000gp/T/ipykernel_3378/855425967.py:53: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  future_dates = pd.date_range(start=

In [14]:
# Display the MSE for each stock ticker
for ticker, mse in mse_dict.items():
    print(f"Stock Ticker: {ticker}, Mean Squared Error: {mse}")

Stock Ticker: AAPL, Mean Squared Error: 5.713258657100702
Stock Ticker: ADBE, Mean Squared Error: 96.3908875100403
Stock Ticker: ADI, Mean Squared Error: 9.955090009149567
Stock Ticker: ALGN, Mean Squared Error: 84.45722746668908
Stock Ticker: AMD, Mean Squared Error: 13.38605656496659
Stock Ticker: AMGN, Mean Squared Error: 12.719893098014568
Stock Ticker: ANSS, Mean Squared Error: 37.69162885529852
Stock Ticker: ASML, Mean Squared Error: 261.6246691910102
Stock Ticker: AVGO, Mean Squared Error: 420.50860733853597
Stock Ticker: BIIB, Mean Squared Error: 16.621219255219703
Stock Ticker: BKNG, Mean Squared Error: 2714.7601010506946
Stock Ticker: CDNS, Mean Squared Error: 18.720096580009915
Stock Ticker: CHKP, Mean Squared Error: 2.8772983305134248
Stock Ticker: CHTR, Mean Squared Error: 70.68113127014459
Stock Ticker: CMCSA, Mean Squared Error: 0.3818627201429725
Stock Ticker: COST, Mean Squared Error: 56.45411949459739
Stock Ticker: CSCO, Mean Squared Error: 0.3946668226128787
Stock Ti

In [15]:
# Display the predictions for each stock ticker
for ticker, future_dates_df in predictions_dict.items():
    print(f"Predictions for {ticker}:")
    print(future_dates_df)


Predictions for AAPL:
                 Close
Date                  
2024-04-30  166.848557
2024-05-31  166.797299
2024-06-30  166.746218
2024-07-31  166.695314
2024-08-31  166.644587
2024-09-30  166.594036
2024-10-31  166.543661
2024-11-30  166.493460
2024-12-31  166.443433
2025-01-31  166.393579
2025-02-28  166.343899
2025-03-31  166.294391
Predictions for ADBE:
                 Close
Date                  
2024-04-30  472.781404
2024-05-31  472.663338
2024-06-30  472.545792
2024-07-31  472.428766
2024-08-31  472.312256
2024-09-30  472.196260
2024-10-31  472.080776
2024-11-30  471.965801
2024-12-31  471.851334
2025-01-31  471.737372
2025-02-28  471.623913
2025-03-31  471.510955
Predictions for ADI:
                 Close
Date                  
2024-04-30  189.567965
2024-05-31  189.208951
2024-06-30  188.852924
2024-07-31  188.499859
2024-08-31  188.149732
2024-09-30  187.802519
2024-10-31  187.458195
2024-11-30  187.116737
2024-12-31  186.778120
2025-01-31  186.442320
2025-02-28  186

In [16]:
# Define the base path as the parent directory of "Notebooks" and "Resources"
base_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))

# Create a directory named "Predictions" within the base path if it doesn't exist
predictions_folder = os.path.join(base_path, "One Year Predictions")
if not os.path.exists(predictions_folder):
    os.makedirs(predictions_folder)

# Save the Predictions results to individual files in the "Predictions" folder
for ticker, future_dates_df in predictions_dict.items():
    prediction_file_path = os.path.join(predictions_folder, f"predictions_{ticker}.csv")
    future_dates_df.to_csv(prediction_file_path)

print(f"Prediction results saved to the folder: {predictions_folder}")

# Create a directory named "MSE" within the base path if it doesn't exist
mse_folder = os.path.join(base_path, "MSE Output")
if not os.path.exists(mse_folder):
    os.makedirs(mse_folder)

# Define the path to save the MSE results file
mse_file_path = os.path.join(mse_folder, "mse_results.csv")

# Save the MSE results to a file in the "MSE" folder
mse_df = pd.DataFrame(list(mse_dict.items()), columns=["Stock Ticker", "Mean Squared Error"])
mse_df.to_csv(mse_file_path, index=False)

print(f"MSE results saved to: {mse_file_path}")


Prediction results saved to the folder: /Users/paulettepetracco/Bootcamp_Code_Challenges/GH_Repositories/Project_4/stockmarket-machinelearningproject/One Year Predictions
MSE results saved to: /Users/paulettepetracco/Bootcamp_Code_Challenges/GH_Repositories/Project_4/stockmarket-machinelearningproject/MSE Output/mse_results.csv
